# Classification multi etiquette par exctraction de features

In [1]:
# Chargement des modules necessaire :

import os

import random
from matplotlib import pyplot as plt

import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras import optimizers
from tensorflow.keras import applications
from tensorflow.keras.models import Model

In [3]:
# On declare les chemins vers les donnees :

Image = 'Data'
        
train_data_dir = 'Data/TRAIN'
validation_data_dir = 'Data/VALIDATION'
test_data_dir = 'Data/TEST'

# Dimmension et path :
img_width, img_height = 224, 224

ImageTRAINCOVID = os.listdir(Image + '/TRAIN/COVID')
ImageTRAINATELECTASIS = os.listdir(Image + '/TRAIN/ATELECTASIS')
ImageTRAINNORMAL = os.listdir(Image + '/TRAIN/NORMAL')
ImageTRAINEFFUSION = os.listdir(Image + '/TRAIN/EFFUSION')
ImageTRAINEMPHYSEMA = os.listdir(Image + '/TRAIN/EMPHYSEMA')
ImageTRAININFILTRATION = os.listdir(Image + '/TRAIN/INFILTRATION')
ImageTRAINPNEUMONIE = os.listdir(Image + '/TRAIN/PNEUMONIE')

ImageVALIDATIONCOVID = os.listdir(Image + '/VALIDATION/COVID')
ImageVALIDATIONATELECTASIS = os.listdir(Image + '/VALIDATION/ATELECTASIS')
ImageVALIDATIONNORMAL = os.listdir(Image + '/VALIDATION/NORMAL')
ImageVALIDATIONEFFUSION = os.listdir(Image + '/VALIDATION/EFFUSION')
ImageVALIDATIONEMPHYSEMA = os.listdir(Image + '/VALIDATION/EMPHYSEMA')
ImageVALIDATIONINFILTRATION = os.listdir(Image + '/VALIDATION/INFILTRATION')
ImageVALIDATIONPNEUMONIE = os.listdir(Image + '/VALIDATION/PNEUMONIE')

ImageTESTCOVID = os.listdir(Image + '/TEST/COVID')
ImageTESTATELECTASIS = os.listdir(Image + '/TEST/ATELECTASIS')
ImageTESTNORMAL = os.listdir(Image + '/TEST/NORMAL')
ImageTESTEFFUSION = os.listdir(Image + '/TEST/EFFUSION')
ImageTESTEMPHYSEMA = os.listdir(Image + '/TEST/EMPHYSEMA')
ImageTESTINFILTRATION = os.listdir(Image + '/TEST/INFILTRATION')
ImageTESTPNEUMONIE = os.listdir(Image + '/TEST/PNEUMONIE')

print('Il y a ' + str(len(ImageTRAINCOVID)) + ' images d\'entrainement de patient covid.') 
print('Il y a ' + str(len(ImageTRAINATELECTASIS)) + ' images d\'entrainement de patient atelectasis.') 
print('Il y a ' + str(len(ImageTRAINNORMAL)) + ' images d\'entrainement de patient sain.') 
print('Il y a ' + str(len(ImageTRAINEFFUSION)) + ' images d\'entrainement de patient effusion.') 
print('Il y a ' + str(len(ImageTRAINEMPHYSEMA)) + ' images d\'entrainement de patient emphyseme.') 
print('Il y a ' + str(len(ImageTRAININFILTRATION)) + ' images d\'entrainement de patient infltration.')
print('Il y a ' + str(len(ImageTRAINPNEUMONIE)) + ' images d\'entrainement de patient pneumonie.')

print('Il y a ' + str(len(ImageVALIDATIONCOVID)) + ' images de validation de patient covid.') 
print('Il y a ' + str(len(ImageVALIDATIONATELECTASIS)) + ' images de validation de patient atelectasis.') 
print('Il y a ' + str(len(ImageVALIDATIONNORMAL)) + ' images de validation de patient non-covid.') 
print('Il y a ' + str(len(ImageVALIDATIONEFFUSION)) + ' images de validation de patient effusion.') 
print('Il y a ' + str(len(ImageVALIDATIONEMPHYSEMA)) + ' images de validation de patient emphyseme.') 
print('Il y a ' + str(len(ImageVALIDATIONINFILTRATION)) + ' images de validation de patient infltration.')
print('Il y a ' + str(len(ImageVALIDATIONPNEUMONIE)) + ' images de validation de patient pneumonie.')

print('Il y a ' + str(len(ImageTESTCOVID)) + ' images test de patient covid.') 
print('Il y a ' + str(len(ImageTESTATELECTASIS)) + ' images test de patient atelectasis.') 
print('Il y a ' + str(len(ImageTESTNORMAL)) + ' images test de patient non-covid.')
print('Il y a ' + str(len(ImageTESTEFFUSION)) + ' images test de patient effusion.') 
print('Il y a ' + str(len(ImageTESTEMPHYSEMA)) + ' images test de patient emphyseme.') 
print('Il y a ' + str(len(ImageTESTINFILTRATION)) + ' images test de patient infltration.')
print('Il y a ' + str(len(ImageTESTPNEUMONIE)) + ' images test de patient pneumonie.')

Il y a 1386 images d'entrainement de patient covid.
Il y a 800 images d'entrainement de patient atelectasis.
Il y a 1106 images d'entrainement de patient sain.
Il y a 800 images d'entrainement de patient effusion.
Il y a 1380 images d'entrainement de patient emphyseme.
Il y a 1348 images d'entrainement de patient infltration.
Il y a 1000 images d'entrainement de patient pneumonie.
Il y a 219 images de validation de patient covid.
Il y a 102 images de validation de patient atelectasis.
Il y a 930 images de validation de patient non-covid.
Il y a 133 images de validation de patient effusion.
Il y a 193 images de validation de patient emphyseme.
Il y a 193 images de validation de patient infltration.
Il y a 333 images de validation de patient pneumonie.
Il y a 240 images test de patient covid.
Il y a 102 images test de patient atelectasis.
Il y a 872 images test de patient non-covid.
Il y a 133 images test de patient effusion.
Il y a 193 images test de patient emphyseme.
Il y a 387 images

In [5]:
# Preprocessing :

# On rescale les images :

datagen = ImageDataGenerator(rescale=1./255)

# On definit la batch size :

batch_size = 32 

# On prepare les tableaux de donnees depuis les images :

train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size)

validation_generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size)

test_generator = datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size)

Found 7816 images belonging to 7 classes.
Found 2102 images belonging to 7 classes.
Found 2260 images belonging to 7 classes.


In [4]:
# Chargement de DENSE121 sans la partie fully-connected avec le reseau convolutif :

model_dense = applications.DenseNet121(include_top=False, weights='imagenet')

In [5]:
# On utilse le model VGG16 pour extraire les features de nos images 
# (on fait recupere la sortie du reseau convolutionnel) :
train_features = model_dense.predict_generator(train_generator, 7816 // batch_size)

validation_features = model_dense.predict_generator(validation_generator, 2102 // batch_size)

# L'opération étant longue on enregistre les features obtenus :
np.save(open('models/trainFeatures_DENSE121_AllDataExtended.npy', 'wb'), train_features) # ecriture en binaire necessaire
np.save(open('models/validationFeatures_DENSE121_AllDataExtended.npy', 'wb'), validation_features) # Idem

In [6]:
# Si l'operation à deja été effectuer on charge les features :

train_features = np.load(open('models/trainFeatures_DENSE121_AllDataExtended.npy', 'rb'))
validation_features = np.load(open('models/validationFeatures_DENSE121_AllDataExtended.npy', 'rb'))

In [19]:
train_labels = np.array([0] * 800 + [1] * 1386 + [2] * 800 + [3] * 1380 + [4] * 1348 + [5] * 1102 + [6] * 992 )

validation_labels = np.array([0] * 102 + [1] * 219 + [2] * 128 + [3] * 192 + [4] * 192 + [5] * 928 + [6] * 319 )

In [20]:
model_top = Sequential()
model_top.add(Flatten(input_shape=train_features.shape[1:]))
model_top.add(Dense(512, activation='relu'))
model_top.add(Dense(126, activation='relu'))
model_top.add(Dropout(0.3))
model_top.add(Dense(7, activation='softmax'))

# On compile :
model_top.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# On affiche :
model_top.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 50176)             0         
_________________________________________________________________
dense_9 (Dense)              (None, 512)               25690624  
_________________________________________________________________
dense_10 (Dense)             (None, 126)               64638     
_________________________________________________________________
dropout_3 (Dropout)          (None, 126)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 7)                 889       
Total params: 25,756,151
Trainable params: 25,756,151
Non-trainable params: 0
_________________________________________________________________


In [21]:
epochs = 20

In [22]:

history = model_top.fit(train_features, train_labels,
        epochs=epochs, 
        batch_size=batch_size,
        validation_data=(validation_features, validation_labels))

Train on 7808 samples, validate on 2080 samples
Epoch 1/20
7808/7808 [==============================] - 150s 19ms/sample - loss: 2.6976 - accuracy: 0.1733 - val_loss: 1.9430 - val_accuracy: 0.0923
Epoch 2/20
7808/7808 [==============================] - 168s 21ms/sample - loss: 1.9304 - accuracy: 0.1767 - val_loss: 1.9425 - val_accuracy: 0.0923
Epoch 3/20
7808/7808 [==============================] - 3037s 389ms/sample - loss: 1.9250 - accuracy: 0.1717 - val_loss: 1.9474 - val_accuracy: 0.0923
Epoch 4/20
7808/7808 [==============================] - 948s 121ms/sample - loss: 1.9234 - accuracy: 0.1766 - val_loss: 1.9456 - val_accuracy: 0.0923
Epoch 5/20
7808/7808 [==============================] - 1727s 221ms/sample - loss: 1.9223 - accuracy: 0.1734 - val_loss: 1.9451 - val_accuracy: 0.0923
Epoch 6/20
7808/7808 [==============================] - 1833s 235ms/sample - loss: 1.9221 - accuracy: 0.1780 - val_loss: 1.9461 - val_accuracy: 0.1053
Epoch 7/20
7808/7808 [=============================

In [24]:
model_top.save('models/DENSE121_Full_10G_224_224')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: models/DENSE121_Full_MAXG_224_224/assets


In [ ]:
# LE modele n'apprend pas 